In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### Change the below to match eda.ipynb

In [2]:
%reload_ext sql
%sql duckdb:///../../movies_data.duckdb

Found pyproject.toml from 'C:\Users\jpjon\Documents\Ploomber\sql\mini-projects\movie-rec-system'

In [3]:
df = %sql select * from movie_genre_data
df = pd.DataFrame(df)
df

Running query in 'duckdb:///../../movies_data.duckdb'

,genre_names,id,original_language,overview,popularity,release_date,title,vote_average,vote_count
0,"Thriller, Action",724209,en,An intelligence operative for a shadowy global...,2813.299,2023-08-09,Heart of Stone,6.9,700
1,"Animation, Action, Adventure",569094,en,"After reuniting with Gwen Stacy, Brooklyn’s fu...",1738.308,2023-05-31,Spider-Man: Across the Spider-Verse,8.5,3696
2,"Action, Adventure, Science Fiction",298618,en,When his attempt to save his family inadverten...,1559.171,2023-06-13,The Flash,7.0,2443
3,"Comedy, Adventure, Fantasy",346698,en,Barbie and Ken are having the time of their li...,1556.661,2023-07-19,Barbie,7.4,3309
4,"Animation, Science Fiction, Action, Adventure",1121575,en,Travel across the galaxy with John Sheridan as...,1519.610,2023-08-15,Babylon 5: The Road Home,7.6,22
...,...,...,...,...,...,...,...,...,...
980,"Action, Comedy, Science Fiction",257344,en,Video game experts are recruited by the milita...,73.242,2015-07-16,Pixels,5.7,7013
981,"Action, Crime, Thriller",273481,en,An idealistic FBI agent is enlisted by a gover...,70.284,2015-09-17,Sicario,7.4,7754
982,Horror,109428,en,Five young friends find the mysterious and fie...,45.753,2013-04-05,Evil Dead,6.5,4190
983,"Action, Adventure, Science Fiction",141052,en,Fuelled by his restored faith in humanity and ...,71.816,2017-11-15,Justice League,6.1,12200


In [4]:
# Create tf-idf matrix for text comparison
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['overview'])

In [5]:
# Compute cosine similarity between all movie-descriptions
similarity = cosine_similarity(tfidf_matrix)
similarity_df = pd.DataFrame(similarity, 
                             index=df.title.values, 
                             columns=df.title.values)
similarity_df.head(10)

,Heart of Stone,Spider-Man: Across the Spider-Verse,The Flash,Barbie,Babylon 5: The Road Home,No Hard Feelings,Meg 2: The Trench,Cobweb,Fast X,Insidious: The Red Door,...,Just Go with It,National Lampoon's Vacation,The Twilight Saga: New Moon,Dawn of the Planet of the Apes,Ghostbusters,Pixels,Sicario,Evil Dead,Justice League,Clash of the Titans
Heart of Stone,1.000000,0.019118,0.022450,0.012476,0.006621,0.021704,0.018739,0.032052,0.035122,0.020683,...,0.016203,0.016691,0.016348,0.004965,0.030542,0.002797,0.015028,0.046110,0.016735,0.035632
Spider-Man: Across the Spider-Verse,0.019118,1.000000,0.100435,0.049998,0.100344,0.029335,0.033016,0.025204,0.079503,0.032446,...,0.021171,0.038298,0.037371,0.030505,0.029616,0.017029,0.042988,0.032074,0.062735,0.057223
The Flash,0.022450,0.100435,1.000000,0.086477,0.090002,0.056523,0.024659,0.091211,0.073312,0.059254,...,0.121867,0.087334,0.074185,0.075716,0.020178,0.027283,0.058568,0.042569,0.153752,0.095988
Barbie,0.012476,0.049998,0.086477,1.000000,0.043066,0.039475,0.045293,0.014550,0.081365,0.076820,...,0.068160,0.070323,0.030241,0.040176,0.026161,0.023496,0.039055,0.043408,0.061784,0.031715
Babylon 5: The Road Home,0.006621,0.100344,0.090002,0.043066,1.000000,0.032574,0.018692,0.033806,0.034673,0.033512,...,0.022676,0.055241,0.041648,0.039612,0.014844,0.017855,0.030133,0.067870,0.034741,0.042810
No Hard Feelings,0.021704,0.029335,0.056523,0.039475,0.032574,1.000000,0.028964,0.078576,0.015296,0.061498,...,0.034739,0.033110,0.066811,0.017082,0.006369,0.006166,0.022047,0.018810,0.026258,0.024873
Meg 2: The Trench,0.018739,0.033016,0.024659,0.045293,0.018692,0.028964,1.000000,0.008631,0.025424,0.061774,...,0.020495,0.012768,0.031308,0.016885,0.011070,0.003848,0.021912,0.034602,0.045212,0.022169
Cobweb,0.032052,0.025204,0.091211,0.014550,0.033806,0.078576,0.008631,1.000000,0.049232,0.038014,...,0.070101,0.036197,0.035468,0.042358,0.003313,0.007505,0.023566,0.033245,0.029389,0.066575
Fast X,0.035122,0.079503,0.073312,0.081365,0.034673,0.015296,0.025424,0.049232,1.000000,0.101126,...,0.050252,0.038631,0.033811,0.053588,0.041993,0.067489,0.054436,0.033403,0.079816,0.050914
Insidious: The Red Door,0.020683,0.032446,0.059254,0.076820,0.033512,0.061498,0.061774,0.038014,0.101126,1.000000,...,0.060049,0.058752,0.042196,0.021866,0.020796,0.016832,0.023166,0.045209,0.040661,0.058364


In [6]:
movie_list = similarity_df.columns.values
def content_movie_recommender(input_movie, similarity_database=similarity_df, movie_database_list= movie_list, top_n=10):

    # movie list 
    movie_list = movie_database_list

    # get movie similarity records
    movie_sim = similarity_database[similarity_database.index == input_movie].values[0]

    # get movies sorted by similarity
    sorted_movie_ids = np.argsort(movie_sim)[::-1]

    # get recommended movie names
    recommended_movies = movie_list[sorted_movie_ids[1:top_n+1]]

    print('\n\nTop Recommended Movies for:', input_movie, 'are:-\n', recommended_movies)

In [7]:
sample_movies = ['Spider-Man: Across the Spider-Verse']
                 
for movie in sample_movies:
    content_movie_recommender(movie)



Top Recommended Movies for: Spider-Man: Across the Spider-Verse are:-
 ['Spider-Man: Into the Spider-Verse' 'Spider-Man'
 'The Amazing Spider-Man 2' 'Spider-Man 3' 'Thor: Ragnarok'
 'Spider-Man: Homecoming' 'Doctor Strange in the Multiverse of Madness'
 'The Amazing Spider-Man' 'Sweet Girl' 'Spider-Man: No Way Home']


### Using both genre and overview columns

In [44]:
df['combined'] = df['overview'] + ' ' + (df['genre_names'] + ', ') * 2 # Duplicate genres to give more weight to it, doesn't really affect the model
df.combined[0]

'An intelligence operative for a shadowy global peacekeeping agency races to stop a hacker from stealing its most valuable — and dangerous — weapon. Thriller, Action, Thriller, Action, '

In [40]:
tfidf_combined = TfidfVectorizer(stop_words='english')
tfidf_matrix_combined = tfidf_combined.fit_transform(df['combined'])

In [41]:
similarity_combined = cosine_similarity(tfidf_matrix_combined)
similarity_df_combined = pd.DataFrame(similarity_combined, 
                                      index=df.title.values, 
                                      columns=df.title.values)

In [42]:
combined_movie_list = similarity_df_combined.columns.values
def content_movie_recommender(input_movie, similarity_database=similarity_df, movie_database_list= combined_movie_list, top_n=10):

    # movie list 
    movie_list = movie_database_list

    # get movie similarity records
    movie_sim = similarity_database[similarity_database.index == input_movie].values[0]

    # get movies sorted by similarity
    sorted_movie_ids = np.argsort(movie_sim)[::-1]

    # get recommended movie names
    recommended_movies = movie_list[sorted_movie_ids[1:top_n+1]]

    print('\n\nTop Recommended Movies for:', input_movie, 'are:-\n', recommended_movies)

In [49]:
sample_movies = ['Spider-Man: Across the Spider-Verse']
                 
for movie in sample_movies:
    content_movie_recommender(movie)



Top Recommended Movies for: Spider-Man: Across the Spider-Verse are:-
 ['Spider-Man: Into the Spider-Verse' 'Spider-Man'
 'The Amazing Spider-Man 2' 'Spider-Man 3' 'Thor: Ragnarok'
 'Spider-Man: Homecoming' 'Doctor Strange in the Multiverse of Madness'
 'The Amazing Spider-Man' 'Sweet Girl' 'Spider-Man: No Way Home']
